In [1]:
# import required modules
import socket
import threading
from tkinter import *
import tkinter as tk
import ast
import pyaudio
import wave
import numpy as np
from PIL import Image, ImageTk
import time
from tkinter import scrolledtext
from tkinter import messagebox
from tkinter import filedialog
import soundfile as sf
import cv2
import random
from IPython.lib.display import Audio
from IPython.core.display import display
import time
from tkinter.filedialog import askopenfile

HOST = '127.0.0.1'
# HOST='172.20.10.2'
PORT = 1234

BUFFER = 2**20
DARK_GREY = '#121212'
MEDIUM_GREY = '#1F1B24'
OCEAN_BLUE = '#464EB8'
WHITE = "white"
FONT = ("Helvetica", 17)
BUTTON_FONT = ("Helvetica", 10)
SMALL_FONT = ("Helvetica", 13)


# Send in Group or Private
# Default = group
Broadcast_To = "Group"
selected_private_user = None
labels = ["Group", "Private"]
active_clients = ['None']
flag_for_update = 0

# for misc files
dict_file_types = {
    "Document" : [('Text Files', '*.txt'), ('Text Files', '*.doc'), ('Text Files', '*.docx'),
                      ('Text Files', '*.xlsx'), ('Text Files', '*xls'), ('Text Files', '*.csv'),
                      ('Text Files', '*.pdf')],
   
    "Photo" : [('Photos', '*.png'), ('Photos', '*.jpeg'), ('Photos', '*.jpg'), ('Photos', '*.gif')],
   
#     "Audio" : [('Videos', '*.wav'), ('Videos', '*.mp3')],
#     "Video" : [('Videos', '*.mp4'), ('Videos', '*.mov'), ('Videos', '*.mkv')]
}

# Creating a socket object
# AF_INET: we are going to use IPv4 addresses
# SOCK_STREAM: we are using TCP packets for communication
client = socket.socket(socket.AF_INET, socket.SOCK_STREAM)


def add_message(message):
    message_box.config(state=tk.NORMAL)
    message_box.insert(tk.END, message + '\n')
    message_box.config(state=tk.DISABLED)

def connect():

    # try except block
    try:

        # Connect to the server
        client.connect((HOST, PORT))
        print("Successfully connected to server")
        add_message("[SERVER] Successfully connected to the server")
    except:
        messagebox.showerror("Unable to connect to server", f"Unable to connect to server {HOST} {PORT}")

    username = username_textbox.get()
    if username != '':
        client.sendall(username.encode())
    else:
        messagebox.showerror("Invalid username", "Username cannot be empty")

    threading.Thread(target=listen_for_messages_from_server, args=(client, )).start()

    username_textbox.config(state=tk.DISABLED)
    username_button.config(state=tk.DISABLED)

   
def send_message():
   
    global Broadcast_To
    global selected_private_user
   
    message = message_textbox.get()
   
    # private msg
    if message!= '' and Broadcast_To == "Private":
        message = "***PRIVATE***" + selected_private_user + "###" + message
        client.sendall(message.encode())
        message_textbox.delete(0, len(message))
   
    elif message != '' and Broadcast_To == "Group":
        client.sendall(message.encode())
        message_textbox.delete(0, len(message))
       
    else:
        messagebox.showerror("Empty message", "Message cannot be empty")


def send_file(msg):
    message = msg
    # private msg
    if message!= '' and Broadcast_To == "Private":
        message = "***PRIVATE***" + selected_private_user + "###" + message
        client.sendall(message.encode())
        message_textbox.delete(0, len(message))
    #
   
    elif message != '' and Broadcast_To == "Group":
        client.sendall(message.encode())
        message_textbox.delete(0, len(message))
       
    else:
        messagebox.showerror("Empty message", "Message cannot be empty")
       
root = tk.Tk()
root.geometry("600x600")
root.title("Messenger Client")
root.resizable(False, False)

root.grid_rowconfigure(0, weight=1)
root.grid_rowconfigure(1, weight=1)
root.grid_rowconfigure(2, weight=4)
root.grid_rowconfigure(3, weight=1)

top_frame = tk.Frame(root, width=600, height=100, bg=DARK_GREY)
top_frame.grid(row=0, column=0, sticky=tk.NSEW)

top_frame1 = tk.Frame(root, width=600, height=25, bg=OCEAN_BLUE)
top_frame1.grid(row=1, column=0, sticky=tk.NSEW)

middle_frame = tk.Frame(root, width=600, height=400, bg=MEDIUM_GREY)
middle_frame.grid(row=2, column=0, sticky=tk.NSEW)

bottom_frame = tk.Frame(root, width=600, height=100, bg=DARK_GREY)
bottom_frame.grid(row=3, column=0, sticky=tk.NSEW)



# ["Photo", "Video", "Audio", "Document", "Others"]
def open_windows_fileExp(*args):
    selected_file_type = other_file_options.get()
    print(selected_file_type)
   
    if selected_file_type in dict_file_types.keys():
        file_types = dict_file_types.get(selected_file_type)
    else:
        file_types = ('All files', '*.*')

    if selected_file_type == "Document":
        file = filedialog.askopenfile(mode='r', filetypes = file_types)
        if file:
            content = file.read()
            file.close()
       
        # create msg
        # msg format -> data = "***DOC***content"
            message = "***DOC***" + content
            send_file(message)
           
           
    elif selected_file_type == "Photo":
        file = filedialog.askopenfilename(filetypes = file_types)
        image_to_send = cv2.imread(file)
       
        # create msg
        # msg format -> data = "***PHOTO***DIMENSIONS***array"
        img_shape = str(image_to_send.shape)
        image_to_send = image_to_send.flatten()
        image_to_send = image_to_send.tolist()
        image_to_send = str(image_to_send)
        message = "***PHOTO***" + img_shape + "***" + image_to_send
        send_file(message)
   
def record_audio():
    CHUNK = 1024
    FORMAT = pyaudio.paInt16
    CHANNELS = 2
    RATE = 44100
    RECORD_SECONDS = 5
    WAVE_OUTPUT_FILENAME = "notification\\output.wav"
    p = pyaudio.PyAudio()

    stream = p.open(format=FORMAT,
                    channels=CHANNELS,
                    rate=RATE,
                    input=True,
                    frames_per_buffer=CHUNK)

    print("* recording")

    frames = []

    for i in range(0, int(RATE / CHUNK * RECORD_SECONDS)):
        data = stream.read(CHUNK)
        frames.append(data)

    print("* done recording")

    stream.stop_stream()
    stream.close()
    p.terminate()
   
    wf = wave.open(WAVE_OUTPUT_FILENAME, 'wb')
    wf.setnchannels(CHANNELS)
    wf.setsampwidth(p.get_sample_size(FORMAT))
    wf.setframerate(RATE)
    wf.writeframes(b''.join(frames))
    wf.close()
   
    # create msg
    # msg format -> audio = "***VOICE***frames"
    frames = str(frames)
    message = "***VOICE***" + frames
    send_file(message)
#

# labels = ["Group", "Private"]

def func():
    global Broadcast_To
    ind = radiobutton_variable.get()
    print(ind)
   
    if ind == 1:
        option.config(state = "normal")
    else:
        option.config(state = "disabled")
   
    Broadcast_To = labels[ind]
    print("***", ind, Broadcast_To)

radiobutton_variable = tk.IntVar()

b1 = tk.Radiobutton(top_frame1, text="Private", variable = radiobutton_variable, value = 1,
               padx = 10, pady = 2.5,command = func)
b1.pack(side=tk.LEFT, padx=10)

b2 = tk.Radiobutton(top_frame1, text="Group", variable = radiobutton_variable, value = 0,
               padx = 10, pady = 2.5, command = func)
b2.pack(side=tk.LEFT, padx=10)
#

# Scroll Down
# def select():
#     selected_private_user = variable_selection.get()
#     print(selected_private_user)


def select_option(*args):
    global variable_selection
    global selected_private_user
    selected_private_user = variable_selection.get()
    print('-------------------')
    print(Broadcast_To)
    print(selected_private_user)
    print('-------------------')    

def update_options():
    global flag_for_update
#     global
    choices = active_clients
    print(choices)
   
    flag_for_update = flag_for_update + 1
   
    newbutt = tk.Button(top_frame1, image = reload_icon, text = "Update", command = update_options)
    new_option_menu = tk.OptionMenu(top_frame1, variable_selection, *choices, command = select_option)
   
    butt.destroy()
    option.destroy()
   
    new_option_menu.pack(side = tk.RIGHT, padx = 30)
    newbutt.pack(side = tk.RIGHT)
   
    if flag_for_update > 1:
        newbutt.destroy()
        new_option_menu.destroy()

       
# choices = ["dhairya", "dhairya1", "dhairya2"]
choices = active_clients

# initial value
variable_selection = tk.StringVar()
variable_selection.set('Choose User')

# option = tk.OptionMenu(top_frame1, variable_selection, *choices, command=select)
# option.pack(side=tk.RIGHT, padx=30)

option = tk.OptionMenu(top_frame1, variable_selection, *choices ,command = select_option)
option.pack(side=tk.RIGHT, padx=30)

option.config(state = "disabled")
reload_icon = tk.PhotoImage(file = "images\\reload.png")
butt = tk.Button(top_frame1, image = reload_icon,text="Update", command = update_options)
butt.pack(side=tk.RIGHT)

#

username_label = tk.Label(top_frame, text="Enter username:", font=FONT, bg=DARK_GREY, fg=WHITE)
username_label.pack(side=tk.LEFT, padx=10)

username_textbox = tk.Entry(top_frame, font=FONT, bg=MEDIUM_GREY, fg=WHITE, width=23)
username_textbox.pack(side=tk.LEFT)

username_button = tk.Button(top_frame, text="Join", font=BUTTON_FONT, bg=OCEAN_BLUE, fg=WHITE, command=connect)
username_button.pack(side=tk.LEFT, padx=15)

# other file types
other_file_choices = ["Photo", "Document"]
other_file_options = tk.StringVar()
other_file_options.set('Send File')
misc_option = tk.OptionMenu(bottom_frame, other_file_options, *other_file_choices, command = open_windows_fileExp)
misc_option.pack(side = tk.LEFT, padx = 5)


message_textbox = tk.Entry(bottom_frame, font=FONT, bg=MEDIUM_GREY, fg=WHITE, width=30)
message_textbox.pack(side=tk.LEFT, padx=10)

mic_icon = tk.PhotoImage(file = "images\\mic1.png")
mic_button = tk.Button(bottom_frame, image = mic_icon, command=record_audio)
mic_button.pack(side=tk.LEFT, padx=5)

message_button = tk.Button(bottom_frame, text="Send", font=BUTTON_FONT, bg=OCEAN_BLUE, fg=WHITE, command=send_message)
message_button.pack(side=tk.LEFT, padx=5)

message_box = scrolledtext.ScrolledText(middle_frame, font=SMALL_FONT, bg=MEDIUM_GREY, fg=WHITE, width=67, height=26.5)
message_box.config(state=tk.DISABLED)
message_box.pack(side=tk.TOP)


# Define the function that will display the image in a pop-up window
def show_image(image_array, geo):
    # Convert the image array to a PIL Image object
    image = Image.fromarray(image_array)

    # Create a new window to display the image
    image_window = tk.Toplevel(root)
    string_geometry = str(geo) + "x" + str(geo)
    image_window.geometry(string_geometry)
   
    # Convert the PIL Image to a PhotoImage object that can be displayed in a Label
    photo = ImageTk.PhotoImage(image)

    # Create a Label widget to display the image
    label = tk.Label(image_window, image=photo)
    label.image = photo
    label.pack()

def listen_for_messages_from_server(client):
    global active_clients
   
    while 1:
       
        message = client.recv(BUFFER).decode('utf-8')
#         display(Audio("notification\\tone.mp3", autoplay=True))

        # updates active_clients list
        if message != '' and "***USERS***" in message:
            # ["", str(list)]
            temp_msg = message.split("***USERS***")[1]
            active_clients = ast.literal_eval(temp_msg)
            print(active_clients)
           
        elif message != '' and "***USERS***" not in message:
            username = message.split("~")[0]
            content = message.split('~')[1]
           
            # recv file -> "***PHOTO***DIMENSIONS***array"
            if "***PHOTO***" in content:
                identity = random.randrange(1, 1000)
                dir_saved_photos = "saved_photos\\"
               
                content = content.split("***PHOTO***")[1]
                content = content.split("***")
                dimensions = ast.literal_eval(content[0])
                data = ast.literal_eval(content[1])
                img = np.array(data)
                img = img.reshape(dimensions)
                img_msg = "Image Received...Saving to Directory..."
               
                path = dir_saved_photos + str(identity) + "RCVD_Photo.png"
                cv2.imwrite(path, img)
                add_message(f"[{username}] {img_msg}")
               
                if dimensions[0] <= 50:
                    geo = 125
                elif dimensions[0] >= 50 and dimensions[0] >= 150:
                    geo = 250
                else:
                    geo = 400
               
                image_final = cv2.imread(path)
                show_image(image_final, geo)
               
            # recv file -> "***DOC***content"
            elif "***Doc***" in content:
                content = content.split("***DOC***")[1]
                add_message(f"[{username}] {content}")
           
            # recv file -> "***VOICE***content"
            elif "***VOICE***" in content:
                content = content.split("***VOICE***")[1]
                content = ast.literal_eval(content)
                img_msg = "Image Received... Saving to Directory... Playing Voice Note..."
               
                identity = random.randrange(1, 1000)
                dir_saved_audio = "saved_voice_notes\\"
                name_saved_audio = dir_saved_audio + str(identity) + "RCVD_VOICE_NOTE.wav"
               
                CHUNK = 1024
                FORMAT = pyaudio.paInt16
                CHANNELS = 2
                RATE = 44100
                RECORD_SECONDS = 5
                p = pyaudio.PyAudio()
               
                wf = wave.open(name_saved_audio, 'wb')
                wf.setnchannels(CHANNELS)
                wf.setsampwidth(p.get_sample_size(FORMAT))
                wf.setframerate(RATE)
                wf.writeframes(b''.join(content))
                wf.close()
               
                add_message(f"[{username}] {img_msg}")
                display(Audio(name_saved_audio, autoplay=True))
               
            else:
                add_message(f"[{username}] {content}")
           
        else:
            messagebox.showerror("Error", "Message recevied from client is empty")

# main function
def main():

    root.mainloop()
   
if __name__ == '__main__':
    main()

C:\Users\jia\AppData\Local\Temp\ipykernel_16248\1519234142.py:19: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display


Successfully connected to server
['user1']
['user1', 'user2']
Photo
